In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
import time
import scipy.linalg as spla
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
import sys

np.set_printoptions(threshold=sys.maxsize)

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

# fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
# fn = '05node_singlephase_balanced_oscillation_03.dss'
# fn = '06node_threephase_radial_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = '06node_threephase_meshed_unbalanced.dss'
#fn = '06node_threephase_unbalanced_VR.dss'
#fn = '02node_threephase_unbalanced_new.dss'
fn = '06node_threephase_radial_unbalanced_transformer.dss'
#fn = '06node_threephase_radial_unbalanced_regulator_simple.dss'
#fn = '06node_threephase_radial_unbalanced_regulator.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
times = [0]
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
der = [0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]
capacitance = [0]

# Pre-generate matrices
t00 = time.time()
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, H_reg, G_reg = basematrices(fn, slackidx, Vslack, None, None)

t0 = time.time()
for _ in range(1):   
    dss.run_command('Redirect ' + fn)
    nnode = len(dss.Circuit.AllBusNames())
    nline = len(dss.Lines.AllNames()) #+ 2

    VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
    STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")

    for i in range(len(times)):
        # Run NR3 with variations in time, DER, and capacitance
        #VNR = #, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        VNR, t2 =NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i], H_reg, G_reg)    
        print(VNR)
            #         VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
#         INR01[i, :, :] = np.reshape(INR, (3, nline))
#         STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
#         SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
#         iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
#         sNR01[i, :, :] = np.reshape(sNR, (3, nnode))
t1 = time.time()

10
7
0
(114, 1)
transformer kvl: 
42
transformer kvl: 
44
transformer kvl: 
46
transformer kvl: 
48
transformer kvl: 
50
transformer kvl: 
52
(0, 114, 114)
(0, 114)


kcl 0
line idx tf
0
1
2
3
4
5
transformer KCL
transformer kcl ???
0 0 1
transformer kcl ???
0 0 3
out lines
0 1 2
transformer kcl ???
0 0 1
transformer kcl ???
0 1 3
out lines
0 1 2
transformer KCL
transformer kcl ???
1 2 1
transformer kcl ???
1 2 3
out lines
1 3 2
transformer kcl ???
1 2 1
transformer kcl ???
1 3 3
out lines
1 3 2
transformer KCL
transformer kcl ???
2 4 1
transformer kcl ???
2 4 3
out lines
2 5 2
transformer kcl ???
2 4 1
transformer kcl ???
2 5 3
out lines
2 5 2
 nr3 0
Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
is there a trnasformer?
is there a trnasformer?
Time to format VNR
0.010998010635375977
Time to solve power flow: 
0.033000946044921875
[[ 1.        +0.j          1.        +0.j          1.00370269+0.00137003j
   1.00370269+0.00137003j 

In [ ]:

dss.run_command('Redirect 06node_threephase_radial_unbalanced_transformer.dss')
tf_no = len(dss.Transformers.AllNames()) - len(dss.RegControls.AllNames()) #number of transformers
vr_no = len(dss.RegControls.AllNames()) #number of voltage regulators


tf_bus = np.zeros((5, tf_no), dtype = int) #tf has in and out bus
vr_bus = np.zeros((5, vr_no), dtype = int) #vr has in and out bus and phase

vr_count = 0 
vr_lines = 0
tf_count = 0
tf_lines = 0
for tf in range(len(dss.Transformers.AllNames())):
    dss.Transformers.Name(dss.Transformers.AllNames()[tf])     
    if dss.Transformers.AllNames()[tf] in dss.RegControls.AllNames(): #start and end bus
        for i in range(2):
            bus = dss.CktElement.BusNames()[i].split('.')
            vr_bus[i, vr_count] = int(dss.Circuit.AllBusNames().index(bus[0]))
        for n in range(len(bus[1:])):   
            vr_lines += 1                    
            vr_bus[int(bus[1:][n]) + 1, vr_count] = int(bus[1:][n])             
        vr_count += 1    
        if len(bus) == 1:
            for n in range(1,4):
                vr_lines += 1
                vr_bus[n+1, vr_count] = n         
    else:
        for i in range(2):
            bus = dss.CktElement.BusNames()[i].split('.')
            tf_bus[i, tf_count] =  int(dss.Circuit.AllBusNames().index(dss.CktElement.BusNames()[i])) #stuff the in and out bus of the tf into an array          
        for n in range(len(bus[1:])):                                 
            tf_bus[int(bus[1:][n]) + 1, tf_count] = int(bus[1:][n])    
        if len(bus) == 1:
            for k in range(1,4):
                tf_lines += 1
                tf_bus[k+1, tf_count] = k          
        tf_count += 1
print(tf_bus)
print(vr_bus)
print(tf_count)
print(tf_lines)
print(vr_count)

In [ ]:
fn = '06node_threephase_radial_unbalanced_regulator_simple.dss'
dss.run_command('Redirect ' + fn)

tf_no = len(dss.Transformers.AllNames()) - len(dss.RegControls.AllNames()) #number of transformers
vr_no = len(dss.RegControls.AllNames()) #number of voltage regulators, independent of phases on each

tf_bus = np.zeros((2, tf_no), dtype = int) #tf has in and out bus
vr_bus = np.zeros((5, vr_no), dtype = int) #vr has in and out bus and phase
vr_lines = 0 #number of phases of voltage regulators

vr_count = 0
tf_count = 0


for tf in range(len(dss.Transformers.AllNames())):
    dss.Transformers.Name(dss.Transformers.AllNames()[tf])     
    if dss.Transformers.AllNames()[tf] in dss.RegControls.AllNames(): #start and end bus
        for i in range(2):
            bus = dss.CktElement.BusNames()[i].split('.')
            vr_bus[i, vr_count] = int(dss.Circuit.AllBusNames().index(bus[0]))
        for n in range(len(bus[1:])):          
            vr_lines += 1
            vr_bus[int(bus[1:][n])+1, vr_count] = int(bus[1:][n])  
        vr_count+=1             
    else:
        for i in range(2):
            tf_bus[i, tf_count] =  int(dss.Circuit.AllBusNames().index(dss.CktElement.BusNames()[i])) #stuff the in and out bus of the tf into an array          
        tf_count += 1
print(vr_lines)

In [ ]:
fn = 'tfregtest.dss'

dss.run_command('Redirect ' + fn)
tf_no = len(dss.Transformers.AllNames()) - len(dss.RegControls.AllNames()) #number of transformers
vr_no = len(dss.RegControls.AllNames()) #number of voltage regulators

tf_bus = np.zeros((2, tf_no), dtype = int) #tf has in and out bus
vr_bus = np.zeros((5, vr_no), dtype = int) #vr has in and out bus and phase

vr_count = 0
vr_lines = 0
tf_count = 0

for tf in range(len(dss.Transformers.AllNames())):
    dss.Transformers.Name(dss.Transformers.AllNames()[tf])     
    if dss.Transformers.AllNames()[tf] in dss.RegControls.AllNames(): #start and end bus
        for i in range(2):
            bus = dss.CktElement.BusNames()[i].split('.')
            vr_bus[i, vr_count] = int(dss.Circuit.AllBusNames().index(bus[0]))
        for n in range(len(bus[1:])):          
            vr_lines += 1
            vr_bus[int(bus[1:][n])+1, vr_count] = int(bus[1:][n])  
        vr_count+=1             
    else:
        for i in range(2):
            tf_bus[i, tf_count] =  int(dss.Circuit.AllBusNames().index(dss.CktElement.BusNames()[i])) #stuff the in and out bus of the tf into an array          
        tf_count += 1

nline = len(dss.Lines.AllNames()) + tf_no + (2 * vr_lines) #should have usual lines, a line for every TF, and 2 lines for every VR
nnode = len(dss.Circuit.AllBusNames())

line_in_idx = range(len(dss.Lines.AllNames()) + tf_no, nline, 2)
line_out_idx = range(len(dss.Lines.AllNames()) + tf_no + 1, nline, 2)
for n in line_in_idx:
    print(n)
print(vr_bus)
print(vr_lines)
print(vr_count)
print(tf_count)
print(tf_bus)

In [ ]:
tf_no = len(dss.Transformers.AllNames()) - len(dss.RegControls.AllNames()) #number of transformers
vr_lines = 0
tf_count = 0
for tf in range(len(dss.Transformers.AllNames())):
    dss.Transformers.Name(dss.Transformers.AllNames()[tf])     
    if dss.Transformers.AllNames()[tf] in dss.RegControls.AllNames(): #start and end bus           
        for i in range(2):
            bus = dss.CktElement.BusNames()[i].split('.')      
        for n in range(len(bus[1:])):          
            vr_lines += 1                  
    else:
        tf_count += 1

nline = len(dss.Lines.AllNames()) #+ tf_no + (2 * vr_count) #should have usual lines, a line for every TF, and 2 lines for every VR
tfVRline = tf_no + (2 * vr_count)
nnode = len(dss.Circuit.AllBusNames())
XNR = np.zeros((2*3*(nnode + nline) + 2*tfVRline,1))
print(vr_lines)
print(tf_count)
print(tfVRline)

In [ ]:
fn = 'tfregtest.dss'
fn ='06_bus_delta_wye.dss'
#fn = '06node_threephase_radial_unbalanced_regulator.dss'
dss.run_command('Redirect ' + fn)

dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
Sbase = 1000000
Vbase = dss.Bus.kVBase() * 1000
Ibase = Sbase / Vbase
print(Ibase)
# print(dss.Circuit.AllBusNames())

# tf_no = len(dss.Transformers.AllNames()) - len(dss.RegControls.AllNames()) #number of transformers
# vr_no = len(dss.RegControls.AllNames()) #number of voltage regulators

# tf_bus = np.zeros((2, tf_no), dtype = int) #tf has in and out bus
# vr_bus = np.zeros((5, vr_no), dtype = int) #vr has in and out bus and phase
# tf_count = 0
# vr_count = 0

# for tf in range(len(dss.Transformers.AllNames())):
#     dss.Transformers.Name(dss.Transformers.AllNames()[tf])     
#     if dss.Transformers.AllNames()[tf] in dss.RegControls.AllNames(): #start and end bus
#         for i in range(2):
#             bus = dss.CktElement.BusNames()[i].split('.')
#             vr_bus[i, vr_count] = int(dss.Circuit.AllBusNames().index(bus[0]))
#         for n in range(len(bus[1:])):            
#             vr_bus[int(bus[1:][n])+1, vr_count] = int(bus[1:][n])  
#         vr_count+=1             
#     else:
#         for j in range(2):
#             print(tf_count)
#             print(j)
#             print(dss.Circuit.AllBusNames().index(dss.CktElement.BusNames()[j]))
#             tf_bus[j, tf_count] =  int(dss.Circuit.AllBusNames().index(dss.CktElement.BusNames()[j])) #stuff the in and out bus of the tf into an array          
#         tf_count += 1

# print(vr_bus)
# nline = len(dss.Lines.AllNames()) + tf_no + (2 * vr_no) #should have usual lines, a line for every TF, and 2 lines for every VR
# print(dss.Lines.AllNames())
# line_in_idx = range(len(dss.Lines.AllNames()) + tf_no, nline, 2)
# line_out_idx = range(len(dss.Lines.AllNames()) + tf_no + 1, nline, 2)
# for n in line_in_idx:
#     print(n)
# print("\n")
# for j in line_out_idx:
#     print(j)


# for l in range(len(dss.Loads.AllNames())):
#     dss.Loads.Name(dss.Loads.AllNames()[l])
#     print(dss.Loads.Name())
#     print(dss.CktElement.YPrim())
#     print(len(dss.CktElement.YPrim()))
  
#     YP = np.asarray(dss.CktElement.YPrim(),dtype='complex')
#     # print(YP)
#     YP = YP[0:-1:2] + 1j*YP[1::2]
#     YP = np.reshape(YP,(int(YP.shape[0]**(1/2)),int(YP.shape[0]**(1/2))))
#     print('Ymatrix - nonpadded')
#     print(YP)
#     print(YP.shape)
#     print("\n")
nnode = len(dss.Circuit.AllBusNames())

# for k1 in range(len(dss.Circuit.AllBusNames())):
#     if k1 == 3 or k1 == 4 or k1 == 5:

#         dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
#         Vtemp = np.asarray(dss.Bus.PuVoltage())
#         Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
#         print(Vtemp)


# a03 = [100, 100, 100]
# a04 = [125, 125, 125]
# a05 = [120, 120, 120]
# kw = {0: a03, 1: a04, 2:a05}

# b03 = [30, 25, 25]
# b04 = [15, 12, 16]
# b05 = [15, 12, 12]
# kvar = {0: b03, 1: b04, 2:b05}


storage = np.array([])
lowest = 5000
pair = [1, 2, 4, 5, 6, 6]
for i in range(0, 200, 20):
    for j in range(0, 100, 10):
        for k in range(0, 200, 20):
            for l in range(0, 100, 10):
                for nn in range(0, 200, 20):
                    for jj in range(0, 100, 10):
                        dss.Loads.Name(dss.Loads.AllNames()[0]) #set the kw and kvar of the 3 wye loads based on for loops
                        dss.Loads.kW(i)
                        dss.Loads.kvar(j)
                        dss.Loads.Name(dss.Loads.AllNames()[1])
                        dss.Loads.kW(k)
                        dss.Loads.kvar(l)
                        dss.Loads.Name(dss.Loads.AllNames()[2])
                        dss.Loads.kW(nn)
                        dss.Loads.kvar(jj)

                        dss.Solution.Solve() #solve this
        
#         for k1 in range(3, 6):
#             dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#             ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
#             Vtemp = np.asarray(dss.Bus.PuVoltage())
#             Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
#             print(Vtemp)
#             absVtemp = np.abs(Vtemp)
#             storage = np.append(storage, Vtemp)
                        for k1 in range(2,5):
                            dss.Lines.Name(dss.Lines.AllNames()[k1]) #go through the relevant lines

                        #     print(dss.Lines.AllNames()[k1])
                            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
                            #Imn = np.asarray(dss.CktElement.Currents())/Ibase
                            Imn = np.asarray(dss.CktElement.Powers()) #format the line powers so they are complex and 3 phase
                        #     print(Imn)
                            Imn = Imn[0:int(len(Imn)/2)]
                            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
                            absItemp = np.abs(Imn) #take the magnitude of each line power
                            storage = np.append(storage, Imn) #add these to an array
                        storage = np.reshape(storage, (3,3)) #shape this array so each row is a line and each column is a phase
                        a = np.mean(np.std(storage, axis = 1)) #take the standard deviation of the phases (normalize) and then the mean of the standard deviations
                        #the metric seems to make sense because you normalize the differences and take the mean of normalized differences
                        
                        if a < lowest: #do some comparison and store the lowest values of a (metric by which matches)
                            lowest = a
                            pair[0] = i
                            pair[1] = j

                            pair[2] = k
                            pair[3] = l

                            pair[4] = nn
                            pair[5] = jj
                        

                        storage = [] #clear the storage (haha!)
            
        #print("\n")
print("PAIR : ")
print(pair)
                
        

# for i in range(len(a03)):
#     for k1 in range(len(dss.Loads.AllNames())):
#         dss.Loads.Name(dss.Loads.AllNames()[k1])
#         if k1 == 0 or k1 == 1 or k1 == 2:
#             dss.Loads.kW(kw[k1][i])
#             dss.Loads.kvar(kvar[k1][i])
           
#     dss.Solution.Solve()
#     for k1 in range(len(dss.Circuit.AllBusNames())):
#         if k1 == 3 or k1 == 4 or k1 == 5:

#             dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#             ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
#             Vtemp = np.asarray(dss.Bus.PuVoltage())
#             Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
#             storage = np.append(storage, np.mean(np.abs(Vtemp)))
#     print('standard deviation ')
#     print(np.std(storage))
#     storage = []
            
# print("\n")

dss.Loads.Name(dss.Loads.AllNames()[0])
dss.Loads.kW(190)
dss.Loads.kvar(10)
dss.Loads.Name(dss.Loads.AllNames()[1])
dss.Loads.kW(190)
dss.Loads.kvar(10)
dss.Loads.Name(dss.Loads.AllNames()[2])
dss.Loads.kW(190)
dss.Loads.kvar(10)
dss.Solution.Solve()


print("\\\\\\\\\\\\\\")
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.Name())
#     print(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Powers())#/Ibase

#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
    
    #print(Imn)
    
    #print(dss.CktElement.Powers())

# for k1 in range(len(dss.Circuit.AllBusNames())):
#     if k1 == 3 or k1 == 4 or k1 == 5:

#         dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
#         Vtemp = np.asarray(dss.Bus.PuVoltage())
#         Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
#         print(Vtemp)

# for k1 in range(len(dss.Circuit.AllBusNames())):
#     if k1 == 3 or k1 == 4 or k1 == 5:

#         dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
#         Vtemp = np.asarray(dss.CktElement.Powers())
#         Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
#         print(Vtemp)


In [ ]:
fn ='06_bus_delta_wye.dss'
dss.run_command('Redirect ' + fn)
for line in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[line])
    print(dss.Lines.Name())
    print(dss.CktElement.Powers())
    print("\n")

Vectorized Solver

OpenDSS Solution

In [ ]:
t0_dss = time.time()
for _ in range(1):
    #dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
#     dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#     dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#     dss.run_command('Redirect 06node_threephase_unbalancedVR.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')
    #dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
    dss.run_command('Redirect 06node_threephase_radial_unbalanced_transformer.dss')
    #dss.run_command('Redirect 06node_threephase_radial_unbalanced.dss')

    VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
    IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
    SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
    kW_list = np.array([])
    kvar_list = np.array([])

    # Store the loads
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        kW_list = np.append(kW_list, dss.Loads.kW())
        kvar_list = np.append(kvar_list, dss.Loads.kvar())

    for i in range(len(times)):
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
        Vbase = dss.Bus.kVBase() * 1000
        Sbase = 1000000.0
        Ibase = Sbase/Vbase
        Zbase = Vbase/Ibase
        print(Ibase)
        print(Zbase)

        # Set slack bus (sourcebus) voltage reference in p.u.
        SlackBusVoltage = 1.000
        dss.Vsources.PU(SlackBusVoltage)

        # Time-varying load
        for k in range(len(dss.Loads.AllNames())):
            dss.Loads.Name(dss.Loads.AllNames()[k])
            dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
            dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))

        dss.Solution.Convergence(0.000000000001)

        # Solve power flow with OpenDSS file
        dss.Solution.Solve()
        if not dss.Solution.Converged:
            print('Initial Solution Not Converged. Check Model for Convergence')
        else:
            print('Initial Model Converged. Proceeding to Next Step.')
            #Doing this solve command is required for GridPV, that is why the monitors
            #go under a reset process
            dss.Monitors.ResetAll()

            # set solution Params
            #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
            dss.Solution.Mode(1)
            dss.Solution.Number(1)
            dss.Solution.StepSize(1)
            dss.Solution.ControlMode(-1)
            dss.Solution.MaxControlIterations(1000000)
            dss.Solution.MaxIterations(30000)
            # Easy process to get all names and count of loads, a trick to avoid
            # some more lines of code
            TotalLoads=dss.Loads.Count()
            AllLoadNames=dss.Loads.AllNames()
            print('OpenDSS Model Compliation Done.')
            print('Iterations: ', dss.Solution.Iterations())
            print('Tolerance: ', dss.Solution.Convergence())

        print('')

        # OpenDSS loads
        for k in range(len(dss.Loads.AllNames())):
            dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
            pattern =  r"(\w+)\."
            load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
            idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) 
            load_ph_arr_temp = [0, 0, 0] 
            for ph1 in range(1, 4): 
                pattern = r"\.%s" % (str(ph1))
                load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
                if load_ph:
                    load_ph_arr_temp[ph1 - 1] = 1
            n= 0
            for m in range(len(load_ph_arr_temp)):
                if load_ph_arr_temp[m] == 1:
                    load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                    n += 2

        print(dss.Solution.Converged())

        # Print number of buses, and bus names
        print(len(dss.Circuit.AllBusNames()))
        print(dss.Circuit.AllBusNames())

        # Print number of loads, and load names
        print(len(dss.Loads.AllNames()))
        print(dss.Loads.AllNames())

        print('')

        VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

        for k1 in range(len(dss.Circuit.AllBusNames())):

            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

        #     print(dss.Circuit.AllBusNames()[k1])
        #     print(dss.Bus.Nodes())

        #     print('puVOTLAGES - LN CARTESIAN')
        #     print(dss.Bus.PuVoltage())

            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]

        #     print(np.asarray(dss.Bus.Nodes(),'int'))

            VDSS[ph,k1] = Vtemp    

        #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


        #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
        #     for k2 in range(len(dss.Bus.Nodes())):
        #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

        print('VDSS\n', np.round(VDSS,decimals=6))


#         IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#         for k1 in range(len(dss.Lines.AllNames())):
#             dss.Lines.Name(dss.Lines.AllNames()[k1])
#         #     print(dss.Lines.AllNames()[k1])
#             ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#             Imn = np.asarray(dss.CktElement.Currents())/Ibase
#         #     print(Imn)
#             Imn = Imn[0:int(len(Imn)/2)]
#         #     print(Imn)
#             Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#         #     print(Imn)
#             IDSS[ph,k1] = Imn
#         #     print('')

#         print('IDSS\n', np.round(IDSS,decimals=6))

#         STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
#         SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#         for k1 in range(len(dss.Lines.AllNames())):
#             dss.Lines.Name(dss.Lines.AllNames()[k1])
#             ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#             Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

#             STXtemp = Sk[0:int(len(Sk)/2)]
#             SRXtemp = Sk[int(len(Sk)/2):]

#             STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
#             SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

#             STXDSS[ph,k1] = STXtemp
#             SRXDSS[ph,k1] = SRXtemp

#         VDSS0[i, :, :] = VDSS
#         IDSS0[i, :, :] = IDSS
#         SRXDSS0[i, :, :] = SRXDSS
#         STXDSS0[i, :, :] = STXDSS


#         print('STXDSS\n', np.round(STXDSS,decimals=6))
#         print('SRXDSS\n', np.round(SRXDSS,decimals=6))

#         print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
#         print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
#         print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))
# t1_dss = time.time()

In [ ]:
print(np.max(np.abs(VDSS - VNR)))

print(VDSS)
print(VNR)

In [ ]:
Vbase = dss.Bus.kVBase() * 1000
Sbase = 1000000.0
Ibase = Sbase/Vbase
Zbase = Vbase/Ibase

In [ ]:
# Set global variables and parameters, and test OpenDSS with network
VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')

#######################################################
###Global variable initialization and error checking###
#######################################################

# Sbase=1
# LoadScalingFactor = 3
# GenerationScalingFactor = 5

# NoiseMultiplyer= 1
# #Set simulation analysis period - the simulation is from StartTime to EndTime
# StartTime = 40600
# EndTime = StartTime + 640
# EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
# #Set hack parameters
# TimeStepOfHack = 160
# PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

# #Set initial VBP parameters for uncompromised inverters
# VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
# VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

# #Set delays for each node
# Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
# #Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
# lpf_meas_vector = (1+0.2*np.random.randn(31))
# lpf_output_vector= (0.1+0.015*np.random.randn(31))

# #Set observer voltage threshold
# ThreshHold_vqvp = 0.06
# adaptive_gain=800


# power_factor=0.9
# pf_converted=tan(acos(power_factor))
# # Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters




# #Error checking of the global variable -- TODO: add error handling here!
# if EndTime < StartTime or EndTime < 0 or StartTime < 0:
#     print('Setup Simulation Times Inappropriately.')
# if NoiseMultiplyer < 0:
#     print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
# dss.run_command('Redirect 02node_threephase_unbalanced.dss')
dss.run_command('Redirect 06node_threephase_radial_unbalanced_transformer.dss')
# dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
# dss.run_command('Redirect IEEE13Nodeckt.dss')

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.00
dss.Vsources.PU(SlackBusVoltage)


# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

print(dss.Settings.VoltageBases())
print(dss.Vsources.BasekV())

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    dss.RegControls.TapNumber(8)
    print(dss.RegControls.TapNumber())
    print('')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        source_voltage = dss.Bus.PuVoltage()
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.Nodes())
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')
    
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    print(dss.Lines.Length())
    Sk = dss.CktElement.Powers()
    print(Sk)
    # print(dss.CktElement.YPrim())
    print(dss.Lines.Phases())
    YPrim = np.asarray(dss.CktElement.YPrim())
    YPrim = YPrim[0::2] + 1j*YPrim[1::2]
    YPrim = YPrim.reshape(2*dss.Lines.Phases(),2*dss.Lines.Phases())
    print(YPrim)
    YPrim = YPrim[0:dss.Lines.Phases(),0:dss.Lines.Phases()]
    print(YPrim)

    print('')

for k1 in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[k1])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print('')

bus1 = 'a01'
bus2 = 'a02'

dss.Circuit.SetActiveBus(bus1)
V1 = np.asarray(dss.Bus.PuVoltage())
print(V1)

dss.Circuit.SetActiveBus(bus2)
V2 = np.asarray(dss.Bus.PuVoltage())
print(V2)

print(V2/V1)
    
# nnode = len(dss.Circuit.AllBusNames())
# nline = len(dss.Lines.AllNames())

# print(source_voltage)
# print(nline)



VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

for k1 in range(len(dss.Circuit.AllBusNames())):

    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

#     print(dss.Circuit.AllBusNames()[k1])
#     print(dss.Bus.Nodes())

#     print('puVOTLAGES - LN CARTESIAN')
#     print(dss.Bus.PuVoltage())

    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

    Vtemp = np.asarray(dss.Bus.PuVoltage())

    Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


#     print(np.asarray(dss.Bus.Nodes(),'int'))

    VDSS[ph,k1] = Vtemp    

#     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


#     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
#     for k2 in range(len(dss.Bus.Nodes())):
#         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

print('VDSS\n', np.round(VDSS,decimals=6))


IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     print(Imn)
    IDSS[ph,k1] = Imn
#     print('')

print('IDSS\n', np.round(IDSS,decimals=6))

STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

    STXtemp = Sk[0:int(len(Sk)/2)]
    SRXtemp = Sk[int(len(Sk)/2):]

    STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

    STXDSS[ph,k1] = STXtemp
    SRXDSS[ph,k1] = SRXtemp

VDSS0[0, :, :] = VDSS
# IDSS0[i, :, :] = IDSS
# SRXDSS0[i, :, :] = SRXDSS
# STXDSS0[i, :, :] = STXDSS
curr = np.array([])
for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print(dss.RegControls.Transformer())
    dss.Transformers.Name(dss.RegControls.Transformer())
    print(np.divide(dss.CktElement.Currents(),Ibase))
    curr = np.append(curr, [dss.CktElement.Currents()])
    print('')

    

IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
    #ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     print(Imn)
    IDSS[ph,k1] = Imn

In [ ]:
# Transformer 06node_threephase_radial_unbalanced_regulator.dss
print(np.max(np.abs(VDSS - VNR)))
print("Difference between corresponding voltages:")
print(np.abs(VDSS-VNR))
print("VDSS: ")
print(VDSS)
print("VNR: ")
print(VNR)

In [ ]:
curr_Reshaped = curr.reshape((3, len(curr)//3))
print(curr_Reshaped)
curr_reshaped_selected = curr_Reshaped[:, 0] + 1j * curr_Reshaped[:, 1]
print("\n")
print(curr_reshaped_selected)
Ibasecurr = curr_reshaped_selected / Ibase

Comparing results between three solutions - DSS and vectorized power flow

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    #print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 

#     print('Complex Current difference')
#     print(np.max(np.abs(IDSS0[i] - INR01[i])))
    
#     print('Complex TX Power difference')
#     print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    
#     print('Complex RX Power difference')
#     print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    
#     print('Voltage Magnitude difference')
#     print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    
#     print('Voltage Angle difference')
#     print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
#     print("\n")

In [ ]:
np.abs(VDSS0[0]-VNR01[0])
print(np.abs(VNR01[0]))

Load comparison between non-vectorized and OpenDSS

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

In [ ]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
for i in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[i])
    print(dss.Loads.kW())
    print(dss.Loads.kvar())
    print(dss.CktElement.Powers())

---

In [ ]:
print(str(dss.Lines.AllNames()) + "\n")
print(dss.Circuit.AllBusNames())
dss.Circuit.AllBusNames()

def linelist(busname):
        in_lines = np.array([])
        out_lines = np.array([])
        for k in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k])
            if busname in dss.Lines.Bus1():
                out_lines = np.append(out_lines, dss.Lines.AllNames()[k])
            elif busname in dss.Lines.Bus2():
                in_lines = np.append(in_lines, dss.Lines.AllNames()[k])
        return in_lines,out_lines
print(linelist(dss.Circuit.AllBusNames()[8]))
print("\n")
print(linelist(dss.Circuit.AllBusNames()[4]))
print("\n")
print("\n")
print(linelist(dss.Circuit.AllBusNames()[0]))
print("\n")
print(linelist(dss.Circuit.AllBusNames()[1]))


In [ ]:
t = -1
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
    return load_order
load_order_list = load_order_f()

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    if t == -1:
        var = 1
    else:
        var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        load_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(load_data[0])
        for ph in range(1, len(load_data)):
            load_kw_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kW() * 1e3 / Sbase / (len(load_data) - 1)
            load_kvar_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kvar() * 1e3 / Sbase / (len(load_data) - 1) 
    return load_kw_arr_ph, load_kvar_arr_ph
load_kw_arr_ph, load_kvar_arr_ph = load_values()
print(load_kw_arr_ph)
print(load_kvar_arr_ph)

In [ ]:

            idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
            for ph in range(1, len(cap_data)):
                caparr[int(cap_data[ph]) - 1, idxbs] += dss.Capacitors.kvar() * 1e3 / Sbase / (len(cap_data) - 1)
        return caparr

In [ ]:
[[0.         0.         0.10384609 0.10384609 0.10384609 0.15576913
  0.        ]
 [0.         0.         0.10384609 0.10384609 0.         0.10384609
  0.05192304]
 [0.         0.         0.10384609 0.10384609 0.10384609 0.
  0.        ]]
[[0.         0.         0.02596152 0.02596152 0.02596152 0.05192304
  0.        ]
 [0.         0.         0.02596152 0.02596152 0.         0.02596152
  0.02596152]
 [0.         0.         0.02596152 0.02596152 0.02596152 0.
  0.        ]]